# Title

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * a
  

###  `sos status`

Command 

```bash
sos status [tasks] -q query
```
checkes the status of tasks. You can specify any number of first characeters of a task to specify a task, for example,

```bash
sos task 7
sos task 77e
sos task 7736e
```
would all work for a task with ID `77e36e7404cf6c2ef7079a09e84a4d6d`, but multiple tasks could be identifies if they share the same leading digits. Actually, 

```
sos task 
```
would match all tasks and list the status of all local tasks.

Option `-q` specifies the task queue to monitor. For example,

```
sos status -q docker
```

would check the status of all tasks on a remote host `docker`. You can monitor the tasks on `docker` on any machine with defined host `docker`.

###  `sos status -v -q`

Option `-v` controls the details of the output of command `sos status`. For example,

```
sos status e7404cf6c2 -v0
```
would print just the status of the task (e.g. `running`).

```
sos task 77e -v1
```
would print the task id and their status

```
77e36e7404cf6c2ef7079a09e84a4d6d    running
77e3c2ef7079a236e7404cf6c2f343d3    completed
```

Option `-v0` and `-v1` could check the status of multiple tasks, as realized by SoS. Some tasks queues have their own task status command and option `-v2` (and upper) will use these commands (if specified) to check the status of the jobs. That is to say

``` bash
sos task 77e36 -v2
```

might return output of a command

```
qstat 18433
```

if the task has been submitted to a cluster named `shark` with a job id `18433`.

If you would like to know more about the tasks,

```bash
sos task 77e36 -v3
```

would list the script the task is running and all variables in abbreviated format, and

```bash
sos task 77e36 -v4
```
would list all variables in complete form.

Finally, using `-q` in combination with `-v` allows you to list the variables used in remote server.

```bash
sos task 77e36 -v4 -q linux
```

## Commands and Options

### `sos remote test`

Command
```
sos remote test
sos remote test hostname
```
goes through all hosts defined in your configuration file and test if they are usable. This command should be used before you use any remote host.

### Option `-q`

Option `-q` can be used by all task-related commands. It specifies a remote server or task queue, with detailed information defined in either global (`~/.sos/config.yml`) or local (`./config.yml`) configuration files. You can also save configurations to other configuration files and specify them using option `-c`. E.g.

```bash
sos run myscript -q shark -c shark.yml
```
uses definition of `shark` defined in `shark.yml`. You can define hosts in any of the configuration files and definitions loaded later override previous definitions. For example, you could define a set of test servers that is identical to definitions in global definition, but with `max_running_jobs` set to 1. You can then use

```
sos run myscript -q shark -c test.yml
```
for test runs and 
```
sos run myscript -q shark
```
for regular execution.

It is also possible for you to specify a host without configuration. Such hosts have default values

```
alias = host
address = host
queue_type = process
path_map = []
shared = []
```

which assumes that the host is a remote machine with identical but not shared file systems and without any task queue.

Finally, if option `-q` is specified without value, the `sos` command will exit with a list of configured hosts so you can use command

```
sos status -q
```
to list all configured queued. Details of each queue will be displayed with option `-v3` or `-v4`.

```
sos status -q -v3
```

### `sos run -q`

The `-q` option of command `sos run` (or `sos-runner`) sets the default task queue for all tasks. For example,

```bash
sos run myscript -q shark
```

would send all tasks in workflow `default` defined in `myscript.sos` to a task queue `shark`. Note that this option does not override option `queue` of steps so you could send some tasks to specific queues and all others to the default queue. 

### `sos dryrun` 

The dryrun running mode is very useful in checking if your scripts are correctly translated and if your machine settings are correct. **It is strongly recommended that you execute your script in dryrun mode before submitting them to remote hosts and/or task queues**.

How tasks are executed depends a bit on your configuration but basically,

1. For local tasks (`process` task engine), the tasks are executed directly with `sos execute task -n` (`-n` for dryrun mode).

2. For direct remote execution (e.g. `sos run script -q server -n` with `queue_type` set to `process` (default)), the tasks will be generated and copied to the remote server, and will be executed with command `sos execute task -n`.

3. For submitting to a PBS task queue (e.g. `sos run script -q pbs -n` with `queue_type` set to `pbs`), the tasks will be generated, copied to remote host. Job files will also be generated according to `job_template` and will be copied to the remote host. However, instead of using `submit_cmd` to submit the job to the PBS queue, **the job script will be executed directly on the head node**. It is therefore important for you to allow the jobs to be run in `dryrun` mode and complete in seconds. Otherwise your system admin would hunt you for running large jobs directly on head nodes.

Dryrun mode allows you to test your workflow on a new server or task queue with the following approach:

1. Define a `host_test` queue that is identical to `host` but with `max_running_jobs` set to 1.
2. Run the script in dryrun mode, which would perform all file synchronization, task translation, submission steps, and execution steps. The only difference is that the scripts are printed instead of executed.
    ```
    sos run script -q host_test -n
    ```
    kill the command with `Ctrl-C` if the scripts are correct.
    
3. Submit one job to the task queue using command
    ```
    sos run script -q host_test
    ```
   kill the command if the task gets started and runs correctly.
4. Submit the rest of the tasks using command
    ```
    sos run script -q host
    ```
    The job submitted at step 3 will be recognized and monitored. 

## Further reading

* 